In [ ]:
# libraries
import pandas as pd
import numpy as np

# 1. Log parsing

In [ ]:
path = r'Data\New_PIT2.txt'

In [ ]:
#====================================================================================
# 1.Get columns from text
def txt_to_df(path):

    # 1.1 Read Data
    f = open(path, 'r')
    content = f.read()
    content_list = content.split('/* JOBSPLIT: ')
    f.close()
    df = pd.DataFrame(content_list, columns=['Text'])

    # 1.2. Define Task type
    df['Task Type'] = df["Text"].str.split().str[0]

    # 1.3 Assign Task ID
    df = df.reset_index(drop=True)
    task_list = [0] 
    for index, row in df.iterrows():
        if (row['Task Type'] == 'TASKSTARTTIME') | (row['Task Type'] == 'JOBENDTIME'):
            task_list.append(task_list[index] +1)
        else:
            task_list.append(task_list[index])   
    df['Task ID'] = task_list[1:]

    # 1.4 Remove unessesary text
    df['Text'] = df[['Text','Task Type']].apply(lambda x: x[0].replace('STEP SOURCE FOLLOWS */\n','') if x[1]=='STEP' 
                                                          else x[0].replace('*/\n',''), axis = 1)
    #.replace("\n", "<br>") 
    return df




df_log = txt_to_df(path)
df_log

In [ ]:
#====================================================================================
# 2. Prepare info about Code

def df_to_code(df):

    #-------------------------------------------
    # 2.1 Extract the Code Syntax
    df_code = df.loc[(df_log['Task Type'].isin(['STEP']))].copy()
    df_code.drop(['Task Type'], inplace=True, axis=1)
    df_code = df_code.rename(columns={'Text': 'Code'})
    df_code = df_code[['Task ID', 'Code']]

    #-------------------------------------------
    # 2.2. Prepare info about Start Time
    df_taskstarttime = df_log.loc[(df_log['Task Type'].isin(['TASKSTARTTIME']))].copy()
    df_taskstarttime['Start Time'] = df_taskstarttime['Text'].str.split().str[1]
    df_taskstarttime['Start Time'] = pd.to_datetime(df_taskstarttime['Start Time'], format='%d%b%Y:%H:%M:%S.%f')
    df_taskstarttime.drop(['Text', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 2.3. Prepare info about Elapsed Time
    df_elapsedtime = df_log.loc[(df_log['Task Type'].isin(['ELAPSED']))].copy()
    df_elapsedtime['Elapsed Time'] = df_elapsedtime['Text'].str.split().str[1].astype('int')/100
    df_elapsedtime.drop(['Text', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 2.4. Prepare info about Procedure Names
    df_procedure = df_log.loc[(df_log['Task Type'].isin(['PROCNAME']))].copy()
    df_procedure['Procedure'] = df_procedure['Text'].str.split().str[1]
    df_procedure.drop(['Text', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 2.5 Output
    df_code = df_code.merge(df_procedure, on=['Task ID'], how = 'outer')\
                     .merge(df_taskstarttime, on=['Task ID'], how = 'outer')\
                     .merge(df_elapsedtime, on=['Task ID'], how = 'outer')
    
    return df_code

df_code = df_to_code(df_log)
df_code

In [ ]:
#====================================================================================
# 3. Extract Edges

def df_to_edges(df):
    
    #-------------------------------------------
    # 3.1 Extract Table info
    # 3.1.a Get data
    df_tables = df.loc[(df['Task Type'].isin(['DATASET', 'OPENTIME', 'TASKSTARTTIME',  'JOBENDTIME']))].copy()   

    # 3.1.b # Extract table name and type
    df_tables['Table'] = np.where(df_tables['Task Type'].isin(['DATASET']),
                                  df_tables['Text'].str.split().str[3],
                                  np.where(df_tables['Task Type'].isin(['OPENTIME']),
                                           df_tables['Text'].str.split().str[1],
                                           ''))
    df_tables['Table type'] = df_tables['Table'].str.split('.').str[-1]                          # Extract table type
    df_tables['Table'] = df_tables['Table'].apply(lambda x: '.'.join(x.split('.')[:-1]))         # Extract table name

    # 3.1.c Define table type (Input, Output, Update)
    df_tables['Dataset type'] = np.where(df_tables['Task Type'].isin(['DATASET']),
                                          df_tables['Text'].str.split().str[1],                  # Define Input, Output, Update tables
                                          '')
    df_tables['Dataset type'] = np.where(df_tables['Task Type'].isin(['OPENTIME']),
                                          df_tables['Dataset type'].shift(),                     # Inherit table type
                                          df_tables['Dataset type'])

    # 3.1.d Add sub-step indicator
    df_tables = df_tables.reset_index(drop=True)
    df_tables['Lag Task ID'] = df_tables['Task ID'].shift()
    df_tables['Lag Dataset type'] = df_tables['Dataset type'].shift()
    sub_step_list = [1]
    for index, row in df_tables.iterrows():
        if (row['Task ID'] != row['Lag Task ID']):
            sub_step_list.append(1)                       # Reset sub-step
        elif ((row['Lag Dataset type'] == 'OUTPUT') & (row['Dataset type'] == 'INPUT')) | \
             ((row['Lag Dataset type'] == 'OUTPUT') & (row['Dataset type'] == 'UPDATE')) | \
             ((row['Lag Dataset type'] == 'UPDATE') & (row['Dataset type'] == 'INPUT')):
            sub_step_list.append(sub_step_list[index] +1) # Increase sub-step
        else:
            sub_step_list.append(sub_step_list[index])    # Keep sub-step
    df_tables['SubTask ID'] = sub_step_list[1:]

    # 3.1.e Correct case for updated tables
    df_tables['Dataset type'] = df_tables['Dataset type'].replace({'UPDATE': 'UPDATE INPUT'})
    df_update = df_tables.loc[df_tables['Dataset type'].isin(['UPDATE INPUT'])].copy()
    df_update['Dataset type'] = df_update['Dataset type'].replace({'UPDATE INPUT': 'UPDATE OUTPUT'})
    df_tables = df_tables.append(df_update).sort_values(['Task ID', 'SubTask ID', 'Dataset type'])
    # 3.1.f Add node index
    node_id_dict = {}
    node_id_list = []
    for index, row in df_tables.iterrows():
        # If update or output:
        if (row['Dataset type'] == 'OUTPUT') | (row['Dataset type'] == 'UPDATE OUTPUT') :
            node_id = str(row['Task ID']) +':'+ str(row['SubTask ID']) +':'+ row['Table']        # create node ID
            node_id_list.append(node_id)                                                         # update column
            node_id_dict[row['Table']] = node_id                                                 # update dict

        # If source table:
        elif row['Table'] not in node_id_dict:        
            node_id = '0:1:' + row['Table']                                                      # create node ID
            node_id_list.append(node_id)                                                         # update column
            node_id_dict[row['Table']] = node_id                                                 # update dict

        # If input:
        else: 
            node_id_list.append(node_id_dict[row['Table']])                                      # use previous node ID 

    df_tables['Node Id'] = node_id_list

    # 3.1.g Remove leftover tables
    df_tables.drop(['Table type','Lag Task ID','Lag Dataset type'], inplace=True, axis=1)

    #-------------------------------------------
    # 3.2. Prepare info about Input tables
    df_input = df_tables.loc[(df_tables['Task Type'].isin(['DATASET']) ) & 
                             (df_tables['Dataset type'].isin(['INPUT', 'UPDATE INPUT']))].copy()
    df_input = df_input.rename(columns={'Table': 'Input Table',
                                        'Node Id': 'Source ID'})
    df_input.drop(['Text', 'Dataset type', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 3.3. Prepare info about Output tables
    df_output = df_tables.loc[(df_tables['Task Type'].isin(['DATASET']) ) & 
                              (df_tables['Dataset type'].isin(['OUTPUT', 'UPDATE OUTPUT']))].copy()
    df_output = df_output.rename(columns={'Table': 'Output Table',
                                          'Node Id': 'Target ID'})
    df_output.drop(['Text', 'Dataset type', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 3.4. Prepare info about Time Calculations
    # 3.4.a Get Data
    df_time = df_tables.loc[df_tables['Task Type'].isin(['OPENTIME', 'TASKSTARTTIME', 'JOBENDTIME'])].copy()

    # 3.4.b Add time
    df_time['Start Time'] = np.where(df_time['Task Type'].isin(['OPENTIME']),
                                     df_time['Text'].str.split().str[2].str.replace('DATE:', ''),
                                     df_time['Text'].str.split().str[1].str.replace('DATE:', ''))
    df_time['Start Time']  = pd.to_datetime(df_time['Start Time'], format='%d%b%Y:%H:%M:%S.%f')

    # 3.4.c Remove columns and rows
    df_time.drop(['Table','Text','Task Type','Dataset type','Node Id'], inplace=True, axis=1)
    df_time = df_time.drop_duplicates(subset=['Task ID', 'SubTask ID'], keep='first')

    # 3.4.d Calculate time for each step
    df_time['Elapsed Time'] = (df_time['Start Time'].shift(-1) - df_time['Start Time']).dt.total_seconds()
    df_time['Elapsed Time'] = df_time['Elapsed Time'].round(2)

    #-------------------------------------------
    # 3.5. Output
    # 3.5.a Merge the results for input and output tables + Elapsed time
    df = df_input.merge(df_output, on=['Task ID', 'SubTask ID'], how = 'outer')\
                 .merge(df_time, on=['Task ID', 'SubTask ID'], how = 'left')\
                 .sort_values(['Task ID', 'SubTask ID'])\
                 .reset_index(drop = True)

    # 3.5.b Fill missing for specific cases
    df['Input Table'] = df['Input Table'].fillna('No Input')
    df['Output Table'] = df['Output Table'].fillna('_null_')

    # 3.5.c Correct Start time
    df['Start Time'] = df['Start Time'].fillna(df['Start Time'].shift()) 
    df['Elapsed Time'] = df['Elapsed Time'].fillna(0)

    # 3.5.d Correct Target node ID
    df['Target ID'] = df['Target ID'].fillna(df['Task ID'].astype('str') +':'+ df['SubTask ID'].astype('str')+ ':No Output') 

    # 3.5.e Correct Source node ID
    df['Source ID'] = df[['Input Table','Task ID','SubTask ID','Source ID']].apply(lambda x: str(int(x[1])-1)+':'+ str(x[2])+  ':No Input' if x[0]=='No Input' 
                                                          else x[3], axis = 1)
    
    # 3.5.f Reorder columns
    df = df[['Task ID','SubTask ID', 'Source ID','Target ID', 'Input Table','Output Table', 'Start Time','Elapsed Time']]
    
    return df

df_edges = df_to_edges(df_log)


df_edges.to_csv('test.csv', sep=';')
df_edges

## Network plot

In [ ]:
# libraries
import networkx as nx

import matplotlib
import matplotlib.cm as cm
from matplotlib import pyplot as plt

import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots

from ipywidgets import interactive, VBox, HBox

In [ ]:
# Create Graph
G = nx.from_pandas_edgelist(df_edges,
                            'Source ID','Target ID',
                            ['Task ID','SubTask ID',
                             'Input Table','Output Table',
                             'Start Time','Elapsed Time'],
                            create_using=nx.DiGraph())

In [ ]:
def get_coords(G):
    # get a list of all nodes 
    list_nodes_full = sorted(list(G.nodes))

    def setdiff_sorted(list_append, list_result):
        ans = np.setdiff1d(list_append, list_result, False).tolist()
        list_result.extend(ans)
        return ans

    # get source nodes 
    source_nodes = sorted([node for node in G.nodes() if G.in_degree(node) == 0])

    list_nodes = []
    for i in range(len(source_nodes)):

        # Define source node
        source = source_nodes[i]

        # Get nodes in depth order
        list_nodes_depth = list(nx.dfs_preorder_nodes(G, source=source, depth_limit=None))

        # Keep nodes not already in list
        list_nodes_unique = setdiff_sorted(list_nodes_depth, list_nodes)

    # Get missed nodes
    list_nodes_miss = setdiff_sorted(list_nodes, list_nodes_full)

    # Create df
    df_nodes = pd.DataFrame(list_nodes, columns =['Node'])
    df_nodes['Task ID'] = df_nodes['Node'].str.split(':').str[0].astype('int')
    df_nodes['SubTask ID'] = df_nodes['Node'].str.split(':').str[1].astype('int')

    # Assign x
    df_nodes['x'] = df_nodes['Task ID']

    # Assign y
    df_nodes = df_nodes.reset_index(drop=True)
    df_nodes['Lag Task ID'] = df_nodes['Task ID'].shift()
    df_nodes['Lag SubTask ID'] = df_nodes['SubTask ID'].shift()
    x_list = [10] 
    for index, row in df_nodes.iterrows():
        if   ( (row['Task ID']-row['Lag Task ID'])==0  & (row['SubTask ID'] != row['Lag SubTask ID']) ):
            x_list.append(x_list[index] + 1)
        elif ( (row['Task ID']-row['Lag Task ID'])<= 0 ):
            x_list.append(x_list[index] + 1)        
        else:
            x_list.append(x_list[index]) 
    df_nodes['y'] = x_list[1:]

    # Coords dict
    coords = df_nodes[['Node', 'x' , 'y']].set_index('Node').T.to_dict('list')
    return coords


# Add coords
coords = get_coords(G)
# Add attributes
nx.set_node_attributes(G, coords, 'coords') 


In [ ]:
# Create Edges
def get_edge_trace(G):
    #====================================================================================        
    # Edge info
    edge_x = []
    edge_y = []
    edge_text = []
    edge_x_text = []
    edge_y_text = []
    edge_color_values = []
    for edge in G.edges():
        #--------------------------------------------------------------------
        # Coords
        x0, y0 = G.nodes[edge[0]]['coords']
        x1, y1 = G.nodes[edge[1]]['coords']
        x_mean = round( (x0+x1)/2, 4)
        y_mean = round( (y0+y1)/2, 4)   
        edge_x.append([x0, x_mean, x1, None])
        edge_y.append([y0, y_mean, y1, None])   
        
        #--------------------------------------------------------------------
        # Hover text
        edge_text.append('Task ID: '+str(G.edges[edge[0],edge[1]]['Task ID']) 
                         +' '+
                         'SubTask ID: '+str(G.edges[edge[0],edge[1]]['SubTask ID'])
                         +'<br> '+
                         'Input: '+str(G.edges[edge[0],edge[1]]['Input Table'])
                         +'<br> '+   
                         'Output: '+str(G.edges[edge[0],edge[1]]['Output Table'])     
                         +'<br> '+                     
                         'Elapsed Time: '+str(G.edges[edge[0],edge[1]]['Elapsed Time']) 
                        )
        edge_x_text.append(x_mean)
        edge_y_text.append(y_mean) 
        
        #--------------------------------------------------------------------
        # Colorbar
        edge_color_values.append(G.edges[edge[0],edge[1]]['Elapsed Time'])
        
    #====================================================================================        
    # b) Edges color
    minima = min(edge_color_values)
    maxima = max(edge_color_values)
    norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap='RdYlGn_r')
    edge_color = []
    for v in edge_color_values:
        rgba = mapper.to_rgba(v)
        edge_color.append( matplotlib.colors.to_hex(rgba, keep_alpha=False))
        
    #====================================================================================        
    # c) Edges lines
    edge_trace = []  
    for i in range(len(edge_color)):
        edge_trace.append(go.Scatter(x=edge_x[i], y=edge_y[i],
                                     mode = 'lines', 
                                     line_shape = 'spline',
                                     line = dict(width=1, 
                                                 dash='dot', 
                                                 color=edge_color[i]),
                                     hoverinfo = 'none',
                                     showlegend=False))
        
    #====================================================================================        
    # Text
    edge_text_trace = go.Scatter(x=edge_x_text, y=edge_y_text, 
                                 # Marker
                                 mode = 'markers', 
                                 marker_symbol = 'hexagram',
                                 marker=dict(showscale=True, 
                                             colorscale='RdYlGn', 
                                             reversescale=True,
                                             size = 8, 
                                             color=edge_color_values,
                                             colorbar=dict(thickness=15,
                                                           title='Execution time (s)',
                                                           xanchor='left',
                                                           titleside='right')
                                            ),
                                 # Text
                                 text = edge_text,  
                                 textposition = 'top center', 
                                 hovertemplate = ' %{text}',
                                 showlegend=False)
    
    return edge_trace, edge_text_trace

edge_trace, edge_text_trace = get_edge_trace(G)

In [ ]:
# Create Nodes
def get_node_trace(G):
    #====================================================================================            
    # 1. Node Attributes
    node_x = []
    node_y = []
    node_table = []
    node_color = []
    node_shape = []
    step_id = []    
    for node in G.nodes():  
        #--------------------------------------------------------------------
        # Node Coords
        x, y = G.nodes[node]['coords']
        node_x.append(x)
        node_y.append(y)        

        #--------------------------------------------------------------------
        # Node name
        table_name = node.split(':')[2]
        node_table.append(table_name)
        
        #--------------------------------------------------------------------
        # Step ID
        step_id.append(node.split(':')[0])        

        #--------------------------------------------------------------------
        # Predesessors
        predecessors = [c.split(':')[2] for c in G.predecessors(node)]

        #--------------------------------------------------------------------
        # Node shape and color
        # a) No Input node
        if table_name == 'No Input':
            node_shape.append("diamond-cross")
            node_color.append("grey")
        # b) No Output node
        elif table_name == 'No Output':
            node_shape.append("square-cross")
            node_color.append("grey") 
        # c) Temporary tables
        elif ('SASTMP' in table_name) | ('SORTTMP' in table_name):
            node_shape.append("star-square-dot")
            node_color.append("silver")    
        # d) Input node
        elif G.in_degree(node) == 0:
            node_shape.append("diamond")
            node_color.append("gold")
        # e) Output node
        elif G.out_degree(node) == 0:
            node_shape.append("square")
            node_color.append("cyan") 
        # f) Updated node            
        elif table_name in predecessors:
            node_shape.append("cross")
            node_color.append("orange")
        # g) Internal nodes
        else:
            node_shape.append("circle")
            node_color.append("blue")

    #====================================================================================        
    # 2. Node plot
    node_trace = go.Scatter(x = node_x, 
                            y = node_y,
                            mode = 'markers',
                            marker=dict(symbol = node_shape, 
                                        color = node_color,
                                        size = 10
                                       ),
                            marker_line_color="black", 
                            marker_line_width=0.5,
                            text = node_table,
                            textposition = 'top center',
                            hovertemplate = ' %{text}')
    return node_trace


node_trace = get_node_trace(G)

## Basic plot

In [ ]:
# Base
fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=False,
                    vertical_spacing=0,
                    specs=[[{"type": "scatter"}], [{"type": "table"}]])

# Network plot
# a) Edges
for i in range(len(edge_trace)):
    fig.add_trace(edge_trace[i], row=1, col=1) 
# b) Nodes
fig.add_trace(node_trace, row=1, col=1)
# c) Text
fig.add_trace(edge_text_trace, row=1, col=1) 

# Table plot
col_list = ['Task ID','Code','Procedure','Elapsed Time']
fig.add_trace(go.Table(columnwidth=[1,13,2,2],
                       header=dict(values=col_list,
                                   font=dict(size=12),
                                   fill = dict(color='#C2D4FF'),
                                   line_color='darkslategray',
                                   align = ['left']),
                       cells=dict(values=[df_code[k].tolist() for k in col_list],
                                  font=dict(size=10),
                                  fill = dict(color='#F5F8FF'),
                                  line_color='darkslategray',
                                  align = ['left'])
                      ), row=2, col=1)
# Update
fig.update_layout(width=1000, height=800,
                  showlegend=False,
                  title_text="SAS Code Process analysis",
                  xaxis=dict(title='Calculation Step', 
                             side='top'),
                  yaxis=dict(showgrid=False, 
                             zeroline=False, 
                             showticklabels=False))


## Dash plot

In [ ]:
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
# Server start
app = dash.Dash()
styles = {'pre': {'border': 'thin lightgrey solid',
                  'overflowX': 'scroll'}
         }

# network plot
fig_net = go.Figure(edge_trace + [node_trace, edge_text_trace]) 
fig_net.update_layout(showlegend=False,
                      xaxis=dict(title='Calculation Step', 
                                 side='top'),
                      yaxis=dict(showgrid=False, 
                                 zeroline=False,
                                 showticklabels=False))

# Dash plot
app.layout = html.Div([
    dcc.Graph(id='basic_graph',
              figure=fig_net),
    html.Div(className='row', 
             children=[
                       # Table
                       dash_table.DataTable(id='table',
                                            columns=[{"name": i, "id": i} for i in df_code.columns],
                                            data=df_code.to_dict('records'),
                                            fixed_rows={'headers': True},
                                            style_table={'height': 450,
                                                         'overflowY': 'scroll',
                                                         'border': 'thin lightgrey solid'},  
                                            style_cell={'textAlign': 'left',
                                                        'whiteSpace': 'pre-line',
                                                        'backgroundColor': 'rgb(153, 204, 255)',
                                                        'color': 'black',
                                                        'minWidth': '10px', 
                                                        'maxWidth': '800px'},
                                            style_header={'fontWeight': 'bold',
                                                          'backgroundColor': 'rgb(0, 0, 153)',
                                                          'color': 'white'}
                                           )
             ])
])

# https://community.plotly.com/t/update-a-dash-datatable-with-callbacks/21382/3
#https://dash.plotly.com/datatable/editable

# Add interactive table
@app.callback(
    Output('table', 'data'),
    Input('basic_graph', 'selectedData'))
def display_relayout_data(selectedData):
    if selectedData is not None:
        print(selectedData)
        x_values = []
        for elements in selectedData['points']:
            x_values.append( elements['x'] )
        print(x_values)
            
            
            
#         x_range_value = points_value['x']
#         print(x_range_value)   
#         print(x_range_value[0])   
#         print(x_range_value[1])   
#         max_value = selectedData['range']['x'][1]
        df = df_code.loc[ df_code['Task ID'].isin(x_values) ]
    else:
        print("No Value selected")
        df = df_code
        
    return df.to_dict('records')

app.run_server(debug=False)